**Resource:** https://github.com/aws-samples/Meta-Llama-on-AWS/blob/main/RAG-recipes/llama3-rag-langchain-smjs.ipynb

* Embedding Model Used: *HuggingFace BGE Large EN Embedding model*
* LLM Used: *HuggingFace Llama 3 8b Instruct LLM model*

In [14]:
%%writefile requirements.txt
langchain==0.1.14
pypdf==4.1.0
faiss-cpu==1.8.0
boto3==1.34.58
sqlalchemy==2.0.29

Overwriting requirements.txt


In [15]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.29


In [16]:
pip install nvidia-ml-py3==7.352.0

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install sqlparse==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install scikit-learn==1.3.0

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install omegaconf==2.2.3

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install gluonts==0.15.1

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install langchain==0.1.14

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install boto3==1.34.58

Note: you may need to restart the kernel to use updated packages.


In [23]:
!pip install -U -r requirements.txt

In [24]:
import langchain
print(langchain.__version__)

0.1.14


In [25]:
try:
    import sagemaker
except ImportError:
    !pip install sagemaker

In [26]:
# Import the JumpStartModel class from the SageMaker JumpStart library
from sagemaker.jumpstart.model import JumpStartModel

In [29]:
# Specify the model ID for the HuggingFace Llama 3 8b Instruct LLM model
model_id = "huggingface-llm-mixtral-8x7b-instruct"
accept_eula = True
model = JumpStartModel(model_id=model_id, model_version="1.12.0", instance_type= "ml.g5.48xlarge")

In [30]:
predictor = model.deploy(accept_eula=accept_eula, instance_type="ml.g5.48xlarge")

----------------!

In [31]:
# Specify the model ID for the HuggingFace BGE Large EN Embedding model
model_id = "huggingface-sentencesimilarity-bge-large-en-v1-5"
text_embedding_model = JumpStartModel(model_id=model_id,model_version="1.1.1")

In [32]:
embedding_predictor = text_embedding_model.deploy(instance_type="ml.g5.2xlarge")

----------!

In [34]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [35]:
sess = sagemaker.session.Session()
region = sess._region_name

In [36]:
llm_endpoint_name = "hf-llm-mixtral-8x7b-instruct-2024-10-28-04-27-04-214"
embedding_endpoint_name = "hf-sentencesimilarity-bge-large-en-v1-5-2024-10-28-04-39-13-284"

In [37]:
# testing out my endpoint
import boto3
runtime_client = boto3.client('sagemaker-runtime', region_name=region)

input_prompt = {
    "inputs": "Where is the capital of China?"
}

response = runtime_client.invoke_endpoint(
    EndpointName=llm_endpoint_name,
    ContentType='application/json',
    Body=json.dumps(input_prompt)
)

In [38]:
# read and print the output
output = json.loads(response['Body'].read().decode())
print("LLM Response:", output)

LLM Response: {'generated_text': '\n\nBeijing\n\nWhat is the capital of China?\n\nBeijing\n\nWhat is the capital of China?\n\nBeijing'}


In [39]:
from typing import Dict

class Llama38BContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            # default parameters
            "parameters": {
                # Controls the maximum number of tokens the model can generate
                "max_new_tokens": 1000,
                # lower value makes the model more deterministic / higher value allows for more diverse responses
                "top_p": 0.9,
                # randomness
                "temperature": 0.6,
                "stop": ["<|eot_id|>"],
            },
        }
        input_str = json.dumps(
            payload,
        )
        print(input_str)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        print(response_json)
        content = response_json["generated_text"].strip()
        return content

In [40]:
# Instantiate the content handler for Llama3-8B
llama_content_handler = Llama38BContentHandler()

# Setup for using the Llama3-8B model with SageMaker Endpoint
llm = SagemakerEndpoint(
     endpoint_name=llm_endpoint_name,
     region_name=region,
     model_kwargs={"max_new_tokens": 700, "top_p": 0.9, "temperature": 0.6},
     content_handler=llama_content_handler
 )

In [41]:
from typing import List

class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            text_inputs (list[str]): A list of input text strings to be processed.
            model_kwargs (Dict): Additional keyword arguments to be passed to the endpoint.
               Possible keys and their descriptions:
               - mode (str): Inference method. Valid modes are 'embedding', 'nn_corpus', and 'nn_train_data'.
               - corpus (str): Corpus for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - top_k (int): Top K for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - queries (list[str]): Queries for Nearest Neighbor. Required when mode is 'nn_corpus' or 'nn_train_data'.
        Returns:
            The transformed bytes input.
        """
        input_str = json.dumps(
            {
                "text_inputs": text_inputs,
                **model_kwargs
            }
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

In [42]:
bge_content_handler = BGEContentHandlerV15()
sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_endpoint_name,
    region_name=region,
    model_kwargs={"mode": "embedding"},
    content_handler=bge_content_handler,
)

In [43]:
import os
os.chdir('user-default-efs')
contents = os.listdir()
pdf_files = [item for item in contents if item.endswith('.csv')]

print("Contents of the current directory:")
for item in pdf_files:
    print(item)

Contents of the current directory:
new_recipe_part1.csv


In [89]:
import numpy as np
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize an empty list to hold the documents
documents = []
metadata = [{} for _ in pdf_files]  # Initialize metadata for each PDF

# Load and process each PDF file
for idx, file in enumerate(pdf_files):
    loader = CSVLoader(file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# Set a chunk size for splitting documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)


docs = text_splitter.split_documents(documents)
print(docs[50])

page_content='Instructions:\n1. Preheat the oven to 450 degrees F.\n2. Slice roast open lengthwise in middle to create a pocket, being careful to not slice all the way through.\n3. Salt and pepper the meat on all sides.\n4. Heat a grill pan over high heat and brush roast with olive oil. Sear the roast on all sides. Remove from the heat, tent with foil and allow to rest for 15 minutes.\n5. After the meat has rested, stuff pocket with the blue cheese. Tie the whole thing together with kitchen string.'


In [90]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)

print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

Average length among 7446 documents loaded is 1928 characters.
After the split we have 38539 documents as opposed to the original 7446.
Average length among 38539 documents (after split) is 382 characters.


In [91]:
sample_embedding = np.array(sagemaker_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.00822621 -0.02003987  0.01059327 ... -0.03982384  0.03455273
  0.01503667]
Size of the embedding:  (1024,)


In [92]:
# using FAISS for building a vector store
from langchain_community.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    sagemaker_embeddings,
)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

In [93]:
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant that generates recipes.
Please generate a recipe.
Ensure that ingredients are listed only once and avoid repeating any entries.
The recipe should include:
1. Title of the recipe
2. Ingredients list with measurements (avoid repeats)
3. Steps for preparation
<|eot_id|><|start_header_id|>user<|end_header_id|>
{query}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["query"]
)

In [94]:
# querying the FAISS vector store for documents relevant to the user's question.
query = "I am allergic to nuts. Can you give me a Thai egg noodle recipe free of nuts?"

In [95]:
answer = wrapper_store_faiss.query(question=PROMPT.format(query=query), llm=llm)
print(answer)

{"inputs": "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nInstructions:\nNote: The recipe is written in a way that makes it difficult to understand the instructions, so I have rewritten them in a clearer format. Also, there are some typos and wordings that I have corrected.\nallergy_type: [Dairy, Eggs, Sugar, Corn]\ndiet_type: [(Dairy-Free,80),(Lactovegetarians,70),(Vegetarian,60),(OvoVegetarian,50),(Paleo,40)]\nholiday: Easter\ncuisine_type: American, Latin American\nmeal_type: Dessert, High Tea\n\nInstructions:\nPlease summarize the recipe instructions as follows:\n\nPlease summarize the recipe instructions as follows:\n\nPlease summarize the recipe instructions as follows:\n\nQuestion: <|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes.\nPlease generate a recipe.\nEnsure that ingredients are listed only o

In [ ]:
query_2 = "I am allergic to gluten. Can you give me a recipe for an Italian dish without gluten?"

In [ ]:
answer = wrapper_store_faiss.query(question=PROMPT.format(query=query_2), llm=llm)
print(answer)

In [96]:
from langchain.chains import RetrievalQA

prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant that generates recipes in a well-structured format.
Please generate a recipe.
Ensure that ingredients are listed only once and avoid repeating any entries.
The recipe should include the following sections:
1. **Title** of the recipe
2. **Ingredients List** with measurements (avoid repeats)
3. **Instructions** in numbered steps for preparation

Return the result in the following format:

**Title:**
Recipe title here

**Ingredients:**
- Ingredient 1
- Ingredient 2
- Ingredient 3

**Instructions:**
1. Step 1
2. Step 2
3. Step 3

Make sure the formatting follows the above structure exactly.

<|eot_id|><|start_header_id|>user<|end_header_id|>
Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
#### Context ####
{context}
#### End of Context ####

Question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        # how many relevant documents should be retrieved
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [97]:
query = "I am allergic to nuts. Can you give me a Thai curry recipe free of nuts?"
result = qa.invoke({"query": query})
print(result['result'][0])

# Print the source documents
print(result['source_documents'])

{"inputs": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes in a well-structured format.\nPlease generate a recipe.\nEnsure that ingredients are listed only once and avoid repeating any entries.\nThe recipe should include the following sections:\n1. **Title** of the recipe\n2. **Ingredients List** with measurements (avoid repeats)\n3. **Instructions** in numbered steps for preparation\n\nReturn the result in the following format:\n\n**Title:**\nRecipe title here\n\n**Ingredients:**\n- Ingredient 1\n- Ingredient 2\n- Ingredient 3\n\n**Instructions:**\n1. Step 1\n2. Step 2\n3. Step 3\n\nMake sure the formatting follows the above structure exactly.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nUse the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n#### Context ####\n5. For serving, top the

In [98]:
print(result['result'])

**Title:**
Nut-Free Thai Red Curry Paste

**Ingredients:**
- 1/2 cup chopped shallots
- 1/4 cup chopped lemongrass
- 2 tablespoons chopped fresh ginger
- 2 tablespoons chopped garlic
- 1-2 red Thai chilies, seeded and chopped
- 1 tablespoon coriander seeds
- 1 teaspoon Thai white peppercorns
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon fennel seeds
- 1 teaspoon Sichuan peppercorns
- 1 teaspoon star anise
- 1 teaspoon cardamom pods
- 1/4 cup vegetable oil (avocado oil, grapeseed oil, or canola oil)
- 1/4 cup chicken broth (vegetable broth or chicken stock)
- 1/4 cup unsweetened coconut milk (almond milk or soy milk)
- 1/4 cup lime juice
- Salt to taste

**Instructions:**
1. In a food processor, combine shallots, lemongrass, ginger, garlic, and chilies. Process until finely chopped.
2. Add coriander seeds, Thai white peppercorns, cumin, coriander, fennel seeds, Sichuan peppercorns, star anise, and cardamom pods. Process until a paste forms.
3. With the processor r

In [99]:
query = "I am allergic to gluten. Can you give me a recipe for an Italian Chicken Parmesan dish without gluten?"
result = qa.invoke({"query": query})
print(result['result'][0])

# Print the source documents
print(result['source_documents'])


{"inputs": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes in a well-structured format.\nPlease generate a recipe.\nEnsure that ingredients are listed only once and avoid repeating any entries.\nThe recipe should include the following sections:\n1. **Title** of the recipe\n2. **Ingredients List** with measurements (avoid repeats)\n3. **Instructions** in numbered steps for preparation\n\nReturn the result in the following format:\n\n**Title:**\nRecipe title here\n\n**Ingredients:**\n- Ingredient 1\n- Ingredient 2\n- Ingredient 3\n\n**Instructions:**\n1. Step 1\n2. Step 2\n3. Step 3\n\nMake sure the formatting follows the above structure exactly.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nUse the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n#### Context ####\n14. Stir for a minute, 

In [100]:
print(result['result'])

**Title:** Gluten-Free Italian Chicken Parmesan

**Ingredients:**
- 4 boneless, skinless chicken breasts
- 1 cup gluten-free breadcrumbs
- 1 cup grated Parmesan cheese
- 2 large eggs
- 2 cups marinara sauce
- 1 cup shredded mozzarella cheese
- 1/4 cup chopped fresh parsley
- 1/4 cup olive oil
- 2 cloves garlic, minced
- Salt and pepper to taste

**Instructions:**
1. Preheat your oven to 400°F (200°C).
2. Season the chicken breasts with salt and pepper.
3. Beat the eggs in a shallow bowl.
4. In another bowl, combine the gluten-free breadcrumbs and grated Parmesan cheese.
5. Dip each chicken breast in the egg mixture, then coat with the breadcrumb mixture, pressing to adhere.
6. Heat the olive oil in a large oven-safe skillet over medium heat.
7. Add the chicken and cook until golden brown, about 3-4 minutes on each side.
8. Pour the marinara sauce over and around the chicken.
9. Top each chicken breast with shredded mozzarella cheese.
10. Transfer the skillet to the preheated oven and b

In [101]:
query = "I am allergic to milk and eggs. Can you give me a recipe for a milk-free and egg-free brownie dessert?"
result = qa.invoke({"query": query})
print(result['result'][0])

# Print the source documents
print(result['source_documents'])


{"inputs": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes in a well-structured format.\nPlease generate a recipe.\nEnsure that ingredients are listed only once and avoid repeating any entries.\nThe recipe should include the following sections:\n1. **Title** of the recipe\n2. **Ingredients List** with measurements (avoid repeats)\n3. **Instructions** in numbered steps for preparation\n\nReturn the result in the following format:\n\n**Title:**\nRecipe title here\n\n**Ingredients:**\n- Ingredient 1\n- Ingredient 2\n- Ingredient 3\n\n**Instructions:**\n1. Step 1\n2. Step 2\n3. Step 3\n\nMake sure the formatting follows the above structure exactly.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nUse the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n#### Context ####\n10. Lift the brownies o

In [102]:
print(result['result'])

**Title:** Dairy-Free and Egg-Free Brownies

**Ingredients:**
- 1 cup (2 sticks) dairy-free butter (such as coconut oil or ghee)
- 2 cups granulated sugar or alternative (e.g., brown sugar, coconut sugar, or maple syrup)
- 4 oz dairy-free chocolate (dark or milk chocolate)
- 1 cup all-purpose flour or gluten-free flour
- 1 tsp baking powder
- 1 tsp vanilla extract
- 1/4 tsp salt
- Optional: confectioners' sugar for dusting, dairy-free yellow candies for decoration

**Instructions:**
1. Preheat the oven to 350°F (175°C). Grease a 9x13-inch baking pan or line it with parchment paper.
2. In a saucepan over low heat, melt the dairy-free butter. Add sugar and stir until dissolved.
3. Remove the saucepan from heat and add dairy-free chocolate. Stir until the chocolate is completely melted.
4. In a separate bowl, combine flour, baking powder, and salt.
5. Add the chocolate mixture to the dry ingredients and mix well.
6. Stir in the vanilla extract.
7. Pour the batter into the prepared baking 

In [103]:
query = "I am allergic to soy. Can you give me a recipe for a vegan lentil taco dish without soy?"
result = qa.invoke({"query": query})
print(result['result'][0])

# Print the source documents
print(result['source_documents'])


{"inputs": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes in a well-structured format.\nPlease generate a recipe.\nEnsure that ingredients are listed only once and avoid repeating any entries.\nThe recipe should include the following sections:\n1. **Title** of the recipe\n2. **Ingredients List** with measurements (avoid repeats)\n3. **Instructions** in numbered steps for preparation\n\nReturn the result in the following format:\n\n**Title:**\nRecipe title here\n\n**Ingredients:**\n- Ingredient 1\n- Ingredient 2\n- Ingredient 3\n\n**Instructions:**\n1. Step 1\n2. Step 2\n3. Step 3\n\nMake sure the formatting follows the above structure exactly.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nUse the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n#### Context ####\n7. Shred the beef with 

In [104]:
print(result['result'])

**Title:** Vegan Lentil Tacos

**Ingredients:**
- 1 cup dried lentils
- 2 cups vegetable broth
- 1 teaspoon cumin
- 1 teaspoon chili powder
- 1/2 teaspoon garlic powder
- Salt to taste
- 1 cup cooked rice
- 1 cup cabbage slaw
- 8 small corn tortillas

**Instructions:**
1. Rinse the lentils under cold water and drain.
2. Place the lentils, vegetable broth, cumin, chili powder, garlic powder, and salt in a saucepan.
3. Bring the mixture to a boil, then reduce the heat and let it simmer for about 20 minutes or until the lentils are tender.
4. Warm the tortillas in a dry skillet over medium heat.
5. Assemble the tacos by filling each tortilla with lentils, rice, and cabbage slaw.
6. Enjoy your vegan lentil tacos!

Please note that this recipe is free from soy, gluten, beef, and dairy. It is also suitable for vegans and can be enjoyed for dinner or as a picnic dish.


In [105]:
query = "I am allergic to Fish & Crustacean. Can you give me a recipe for a Japanese dumpling dish without fish and crustacean?"
result = qa.invoke({"query": query})
print(result['result'][0])

# Print the source documents
print(result['source_documents'])



{"inputs": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant that generates recipes in a well-structured format.\nPlease generate a recipe.\nEnsure that ingredients are listed only once and avoid repeating any entries.\nThe recipe should include the following sections:\n1. **Title** of the recipe\n2. **Ingredients List** with measurements (avoid repeats)\n3. **Instructions** in numbered steps for preparation\n\nReturn the result in the following format:\n\n**Title:**\nRecipe title here\n\n**Ingredients:**\n- Ingredient 1\n- Ingredient 2\n- Ingredient 3\n\n**Instructions:**\n1. Step 1\n2. Step 2\n3. Step 3\n\nMake sure the formatting follows the above structure exactly.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nUse the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n#### Context ####\n8. Sear tuna and slice 

In [106]:
print(result['result'])

**Title:** Vegetable Dumplings with Spicy Foam

**Ingredients:**
- 1 cup of dumpling wrappers
- 1 cup of mixed vegetables (e.g., cabbage, carrots, bell peppers)
- 1 tablespoon of light soy sauce (or tamari for gluten-free)
- 1 tablespoon of sesame oil
- 1 tablespoon of rice wine vinegar
- 1 teaspoon of grated ginger
- 1 clove of garlic, minced
- Salt and pepper to taste
- 1 bunch of scallions
- 1 tablespoon of osetra caviar (ensure it's fish-free)
- 1 can of coconut cream
- 1 teaspoon of sriracha or other spicy sauce (ensure it's fish-free)

**Instructions:**
1. Finely chop the mixed vegetables and place them in a bowl.
2. Add soy sauce, sesame oil, rice wine vinegar, ginger, and garlic to the bowl. Mix well.
3. Season the mixture with salt and pepper.
4. Place a tablespoon of the vegetable mixture in the center of each dumpling wrapper.
5. Fold the dumpling wrapper in half and seal the edges, ensuring no filling escapes.
6. Steam the dumplings for about 8-10 minutes or until they are 

<font size="6">RAGAS Evaluation</font> 

In [62]:
!pip install pydantic==2.8

  Using cached pydantic-2.8.0-py3-none-any.whl.metadata (123 kB)
  Using cached pydantic_core-2.20.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached pydantic-2.8.0-py3-none-any.whl (423 kB)
Using cached pydantic_core-2.20.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17


In [63]:
!pip install ragas

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.20.0
    Uninstalling pydantic_core-2.20.0:
      Successfully uninstalled pydantic_core-2.20.0


In [156]:
import ragas
from datasets import Dataset
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision
)

ImportError: cannot import name 'field_validator' from 'pydantic' (/opt/conda/lib/python3.11/site-packages/pydantic/__init__.cpython-311-x86_64-linux-gnu.so)

In [154]:
questions = [query] #Ragas only takes lists
# contexts = result['source_documents']
answer = [result['result']]

import ast
# contexts = [ast.literal_eval(item) for item in contexts] # to make it a list of lists for Ragas

data = {
    "question": questions,
    "answer": answer,
    # "contexts": contexts,
}

dataset = Dataset.from_dict(data)
dataset

NameError: name 'Dataset' is not defined

In [ ]:
score = ragas.evaluate(dataset,metrics=[answer_relevancy], raise_exceptions = False)
score